## Import Dependencies

In [60]:
#Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import pandas as pd

## Connect to PyMongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection = db.news

## Latest News

In [11]:
# Set up splinter for news section
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://redplanetscience.com/'
browser.visit(url)

#scrape webpage for container
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_='list_text')

#print title and subtitle
for result in results:
    #scrape title
    title = result.find('div', class_='content_title').text
    
    #scrape subtitle
    subtitle = result.find('div', class_='article_teaser_body').text
    
    #print data
    print("---------------")
    print(title)
    print(subtitle)
    
    # Dictionary to be inserted into MongoDB
    post = {
        'title': title,
        'subtitle': subtitle,
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\17738\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [15]:
# Display the first title record from above
articles = db.news.find()
news_title = articles[0]["title"]
print(news_title)

NASA's Mars 2020 Rover Completes Its First Drive


In [16]:
#Display the first subtitle record from above
news_p = articles[0]["subtitle"]
print(news_p)

In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-meter) increments over small ramps.


In [17]:
browser.quit()

## Featured Image

In [27]:
# Set up splinter for image section
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://spaceimages-mars.com/'
browser.visit(url)

#scrape webpage for full img link
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.links.find_by_partial_text('FULL IMAGE').click()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\17738\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [44]:
results = soup.find('a', class_='showimg fancybox-thumbs')["href"]
featured_image_url = url + results
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [43]:
browser.quit()

## Fact Table

In [66]:
#using pandas
# Webpage url                                                                                                               
url = 'https://galaxyfacts-mars.com/'

# Extract tables
dfs = pd.read_html(url)
mars_df = dfs[1]
mars_df

mars_table_html = mars_df.to_html()
mars_table_html = mars_table_html.replace('\n', '')
mars_table_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    

## Mars Hemisphere Section

In [90]:
# Set up splinter for hemisphere section
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://marshemispheres.com/'
browser.visit(url)

#scrape webpage for full img link and hemisphere name
hemisphere_image_urls = []
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_='item')
for result in results:
    #find title with first url
    title = result.find('h3').text
    
    #click the title link to find full res image
    browser.click_link_by_partial_text(title)
    
    #save the new url to get full res image url
    new_html = browser.html
    new_soup = BeautifulSoup(new_html, 'html.parser')
    partial_url = new_soup.find('img', class_="wide-image")['src']
    img_url = url + partial_url
    print(title)
    print(img_url)
    print("----------")
    
    #go back to previous page for next title and image
    browser.back()
    
    # Dictionary to be inserted into list
    post = {
        'title': title,
        'img_url': img_url,
    }
    hemisphere_image_urls.append(post)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\17738\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
----------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
----------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
----------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
----------


In [91]:
browser.quit()

In [92]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]